In [4]:

import pandas as pd
df = pd.read_csv("/content/parkinsons.data")
df.info()
df.head()
df.drop(["name"],axis = 1, inplace = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [ ]:

df.drop_duplicates(inplace=True)


In [5]:
from sklearn.preprocessing import RobustScaler
import pandas as pd

# Separate features (X) and target (y)
X = df.drop('status', axis=1)
y = df['status']

# Initialize RobustScaler
scaler = RobustScaler()

# Fit and transform the features
X_scaled = scaler.fit_transform(X)

# Create a new DataFrame with the scaled features
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Display the first 5 rows of the scaled features
# display(X_scaled_df.head())

# Combine X_scaled_df and y into a single DataFrame
df_scaled = pd.concat([X_scaled_df, y], axis=1)

# Display the first few rows of the combined DataFrame
display(df_scaled.head())

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE,status
0,-0.441707,-0.207369,-0.526096,0.742638,1.00,0.551724,1.360382,0.552147,0.971469,1.017370,...,0.751248,0.530053,-0.178988,-0.488228,1.067800,0.646652,0.453808,-0.111831,0.784236,1
1,-0.404773,-0.304210,0.170544,1.213828,1.25,0.988506,2.038186,0.989264,1.794668,2.009926,...,1.543816,0.387015,-0.510421,-0.226126,1.116420,1.172214,1.112711,0.233233,1.511499,1
2,-0.492477,-0.500520,0.129918,1.423816,1.50,1.351724,2.443914,1.355828,1.373246,1.295285,...,1.229617,0.072534,-0.243981,-0.397332,1.182613,0.910097,0.879910,-0.035868,1.199543,1
3,-0.492569,-0.424857,0.126526,1.288092,1.50,1.158621,2.047733,1.159509,1.494387,1.468983,...,1.368552,0.094852,-0.245172,-0.366813,1.113138,1.142077,1.098953,0.081927,1.514105,1
4,-0.502723,-0.381093,0.113768,2.023047,2.00,1.862069,3.050119,1.866564,1.930776,1.801489,...,1.839712,0.304844,-0.414462,-0.472751,1.161907,1.405424,0.149003,-0.054626,1.872110,1


In [6]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import os
import joblib

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 5],
    'subsample': [0.7, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.9, 1.0]
}

xgb_classifier_tuned = XGBClassifier(random_state=42, eval_metric='logloss')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(estimator=xgb_classifier_tuned,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=skf,
                           verbose=1,
                           n_jobs=-1)

grid_search.fit(X_scaled_df, y)

print(f"Best hyperparameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

XG_model = XGBClassifier(
    random_state = 42,
    eval_metric = "logloss",
    colsample_bytree=grid_search.best_params_['colsample_bytree'],
    learning_rate=grid_search.best_params_['learning_rate'],
    max_depth=grid_search.best_params_['max_depth'],
    n_estimators=grid_search.best_params_['n_estimators'],
    subsample=grid_search.best_params_['subsample']
)
XG_model.fit(X_scaled_df,y)

os.makedirs("/content/XG_model",exist_ok= True)

joblib.dump(XG_model, "/content/XG_model/xgboost_model_new.joblib")
joblib.dump(scaler,"/content/scaler.joblib")

print("XGBoost model saved to /content/XG_model/xgboost_model_new.joblib")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best hyperparameters found: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}
Best cross-validation accuracy: 0.9487
XGBoost model saved to /content/XG_model/xgboost_model_new.joblib
